# Readme

- author: laplaciannin102(Kosuke Asada)
- date: 2021/01/12
- latest version: 0.1.1

---

## How to install

```shell
pip install copy_excel_format
```

---

## 概要

### 何をするプログラム？

- たくさんのテーブル(DataFrameを想定)をたくさんの書式付きexcelシートとして出力する.

### 注意事項

- xlwingsを使用して並列処理する場合はexcelのインストール(Office)が必要.
- テスト等きちんと出来ていないのでバグがある可能性あり.

---

## 入出力

### 入力

- 複数のpandas.DataFrame.
- 書式のテンプレートとして使用したいexcelシート.

### 出力

- **書式付き**, **テーブルの値が入力済み**のシートが複数あるexcelファイル.
  - 書式はテンプレートexcelファイルのもの.
  - テーブルの値はpandas.DataFrameのもの.

---

## 参考

- [openpyxlライブラリ](https://pypi.org/project/openpyxl/)
- [xlwingsドキュメント](https://docs.xlwings.org/ja/latest/#)

- [PythonでExcelシートを別のワークブックにコピーする方法](https://www.it-swarm-ja.tech/ja/python/python%E3%81%A7excel%E3%82%B7%E3%83%BC%E3%83%88%E3%82%92%E5%88%A5%E3%81%AE%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%96%E3%83%83%E3%82%AF%E3%81%AB%E3%82%B3%E3%83%94%E3%83%BC%E3%81%99%E3%82%8B%E6%96%B9%E6%B3%95/831860805/)

# Load modules

## copy-excel-format module

In [ ]:
# copy_excel_format
import copy_excel_format as cef

## other modules

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import random
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
gc.collect()

# Configures

## random seed

In [ ]:
np.random.seed(57)
random.seed(57)

# Constants

## paths

## directory paths

In [ ]:
input_path = './input/'
output_path = './output/'
interm_path = './intermediate/'

## file paths

In [ ]:
input_template_excel_path = input_path + 'input_template_excel_sample.xlsx'
input_header_csv_path = input_path + 'input_header_df_sample.csv'

# Load sample files

In [ ]:
cef.load_sample_files()

# Functions

## get_sample_df

In [ ]:
header_df = pd.read_csv(input_header_csv_path)
header_df.head()

In [ ]:
def get_sample_df(n_rows=10, header_df=header_df):
    """
    sampleデータを作成する関数.
    
    Args:
        n_rows: int, optional(default=10)
            データ部分のDataFrameの行数.
        
        header_df: pandas.DataFrame
            ヘッダー部分のDataFrame
    """
    col1_samples = ['hoge', 'fuga', 'poyo', 'gray', None]
    
    sample_df = pd.DataFrame()
    sample_df['No.'] = range(n_rows)
    sample_df['No.'] = sample_df['No.'] + 1
    sample_df['date'] = [datetime(2020, 11, 20) + relativedelta(days=jj*7) for jj in range(n_rows)]
    sample_df['col1'] = random.choices(col1_samples, k=n_rows)
    sample_df['col2'] = np.random.randint(0, 10, size=n_rows)
    sample_df['col3'] = np.random.randint(100, 200, size=n_rows)
    sample_df['col4'] = random.choices(col1_samples, k=n_rows)
    sample_df['col5'] = np.random.randint(0, 10, size=n_rows)
    sample_df['col6'] = np.random.randint(100, 200, size=n_rows)
    
    # headerをつける
    tmp_name = random.choice(['hoge', 'fuga', 'poyo'])
    tmp_header_df = header_df.copy()
    tmp_header_df = tmp_header_df.replace('name: <name>', 'name: ' + tmp_name)
    
    sample_df = tmp_header_df.append(sample_df)
    
    return sample_df

### get_sample_df example

In [ ]:
sample_df = get_sample_df()
sample_df.head(3)

In [ ]:
sample_df.tail()

# excel書式コピー準備

## テンプレートのexcelパスとシート名とDataFrameをセット

In [ ]:
# DataFrameの数. シート数も同じ数.
n_df = 10

# CopyExcelInfoHolderオブジェクトインスタンスのリスト
ceih_list = []

# ceih_listというCopyExcelInfoHolderオブジェクトインスタンスのリストを作成しておく
for ii in range(n_df):
    
    tmp_sheet_name = 'sheet' + str(ii+1).zfill(3)
    tmp_df = get_sample_df(
        n_rows = np.random.randint(10, 28)
    )

    ceih = cef.CopyExcelInfoHolder(
        template_excel_path = input_template_excel_path,
        template_sheet_name = 'blank_template',
        output_sheet_name = tmp_sheet_name,
        df = tmp_df
    )
    
    ceih_list += [ceih]
    
    del ceih
    gc.collect()

In [ ]:
# ceih_listというCopyExcelInfoHolderオブジェクトインスタンスのリストを作成しておく
ceih_list[:3]

In [ ]:
# ceih_listの中身を表示
[print('template_excel_path:{}'.format(ii.template_excel_path)) for ii in ceih_list[:3]]

In [ ]:
# ceih_listの中身を表示
[print('template_sheet_name:{}'.format(ii.template_sheet_name)) for ii in ceih_list[:3]]

In [ ]:
# ceih_listの中身を表示
[print('output_sheet_name:{}'.format(ii.output_sheet_name)) for ii in ceih_list[:3]]

In [ ]:
# ceih_listの中身を表示
[print('*' * 80 + '\ndf.head(3):{}'.format(ii.df.head(3)) + '\n' + '*' * 80 + '\n\n') for ii in ceih_list[:3]]

In [ ]:
# ceih_listの中身を表示
[print('*' * 80 + '\ndf.tail(3):{}'.format(ii.df.tail(3)) + '\n' + '*' * 80 + '\n\n') for ii in ceih_list[:3]]

In [ ]:
print(len(ceih_list))

# excel書式コピーを直列で実行

## 出力ファイル名定義

In [ ]:
output_excel_path = output_path + 'output_excel_sample.xlsx'
output_excel_path

## 実行

In [ ]:
start = time.time()

In [ ]:
# copy_excel_format関数を実行
cef.copy_excel_format(
    ceih_list = ceih_list,
    output_excel_path = output_excel_path,
    cef_manual_set_rows = None,
    cef_force_dimension_copy = False,
    cef_debug_mode = True,
    write_index = False,
    write_header = False,
    copy_values = False
)

## 処理時間確認

In [ ]:
cef.get_elapsed_time(start)

# excel書式コピーを並列で実行1(1つの関数で実行)

## cpu数確認

In [ ]:
print('cpu_count:{}'.format(str(os.cpu_count())))

## 出力ファイル名定義

In [ ]:
output_excel_path = output_path + 'output_excel_sample_parallel001.xlsx'
output_excel_path

## 一時的出力ディレクトリ名定義

In [ ]:
tmp_output_excel_dir_path = interm_path + 'tmp_output_excel/'
tmp_output_excel_dir_path

## 実行

In [ ]:
start = time.time()

In [ ]:
# copy_excel_format関数の並列版を実行
cef.copy_excel_format_parallel(
    ceih_list = ceih_list,
    output_excel_path = output_excel_path,
    tmp_output_excel_dir_path = tmp_output_excel_dir_path,
    parallel_method = 'multiprocess',
    n_jobs = None,
    copy_sheet_method = 'xlwings',
    sorted_sheet_names_list = None,
    del_tmp_dir = True,
    n_seconds_to_sleep = 1,
    cef_manual_set_rows = None,
    cef_force_dimension_copy = False,
    cef_debug_mode = True,
    write_index = False,
    write_header = False,
    copy_values = False
)

## 処理時間確認

In [ ]:
cef.get_elapsed_time(start)

# excel書式コピーを並列で実行2(2つの関数に分けて実行)

## 出力ファイル名定義

In [ ]:
output_excel_path = output_path + 'output_excel_sample_parallel002.xlsx'
output_excel_path

## 一時的出力ディレクトリ名定義

In [ ]:
tmp_output_excel_dir_path = interm_path + 'tmp_output_excel/'
tmp_output_excel_dir_path

## 実行

In [ ]:
start = time.time()

In [ ]:
# 並列処理を行い, 一時的な書式設定済みのexcelファイルを出力する.
cef.output_temporary_excel_parallel(
    ceih_list = ceih_list,
    tmp_output_excel_dir_path = tmp_output_excel_dir_path,
    parallel_method = 'multiprocess',
    n_jobs = None,
    cef_manual_set_rows = None,
    cef_force_dimension_copy = False,
    cef_debug_mode = True,
    write_index = False,
    write_header = False,
    copy_values = False
)

In [ ]:
# 一時的に出力した複数のexcelファイルをまとめて複数シートを持つ1つのexcelファイルとする.
cef.copy_excel_format_from_temporary_files(
    ceih_list = ceih_list,
    output_excel_path = output_excel_path,
    tmp_output_excel_dir_path = tmp_output_excel_dir_path,
    copy_sheet_method = 'xlwings',
    sorted_sheet_names_list = None,
    del_tmp_dir = True,
    n_seconds_to_sleep = 1
)

## 処理時間確認

In [ ]:
cef.get_elapsed_time(start)